# 네이버 영화[상영작/예정장] 상영영화 크롤링

Front 요구 사항 : <br>

data class Movie( <br>
        val title: String, <br>
        val genre_ids: List Integer, <br>
        val vote_average: Double, <br>
        var poster_path: String, <br>
        var id: Int <br>
)

In [46]:
import requests
from bs4 import BeautifulSoup
import re

In [2]:
res = requests.get("https://movie.naver.com/movie/running/current.nhn")

In [4]:
soup = BeautifulSoup(res.content, 'lxml')

In [6]:
body = soup.body

In [14]:
li_current_movie = body.select("div#content > div > div.obj_section > div.lst_wrap > ul > li")

In [20]:
li_current_movie[0].select("div > a > img")

[<img alt="담보" onerror="this.src='https://ssl.pstatic.net/static/movie/2012/09/dft_img99x141.png'" src="https://movie-phinf.pstatic.net/20200929_240/1601370752754trKxu_JPEG/movie_image.jpg?type=m99_141_2"/>]

In [22]:
# 영화 포스터 주소

for current_movie in li_current_movie:
    print(current_movie.find("img")["src"])

https://movie-phinf.pstatic.net/20200929_240/1601370752754trKxu_JPEG/movie_image.jpg?type=m99_141_2
https://movie-phinf.pstatic.net/20200916_163/16002324767974gUSf_JPEG/movie_image.jpg?type=m99_141_2
https://movie-phinf.pstatic.net/20200929_185/1601357130092TtONR_JPEG/movie_image.jpg?type=m99_141_2
https://movie-phinf.pstatic.net/20200728_86/15959209130236RePy_JPEG/movie_image.jpg?type=m99_141_2
https://movie-phinf.pstatic.net/20200902_168/15990280770197W1w5_JPEG/movie_image.jpg?type=m99_141_2
https://movie-phinf.pstatic.net/20200915_138/1600148005503cNDi5_JPEG/movie_image.jpg?type=m99_141_2
https://movie-phinf.pstatic.net/20200909_147/1599614480817jHOPg_JPEG/movie_image.jpg?type=m99_141_2
https://movie-phinf.pstatic.net/20200923_268/1600823900967BHEbW_JPEG/movie_image.jpg?type=m99_141_2
https://movie-phinf.pstatic.net/20200917_291/1600333596388McCK9_JPEG/movie_image.jpg?type=m99_141_2
https://movie-phinf.pstatic.net/20200924_161/1600923522860gpssR_JPEG/movie_image.jpg?type=m99_141_2
h

In [24]:
test = li_current_movie[0].find("dl", attrs={"class":"lst_dsc"}) # 영화 상세 정보
# title
# genre_ids
# vote_average
# id(code)

In [58]:
# 연령
# test = li_current_movie[0].find("dl", attrs={"class":"lst_dsc"})
age = re.search(r'[0-9]*', test.select_one("dt.tit > span").text)[0]

In [79]:
# title
title = test.select_one("dt.tit > a").text

# movie_id
movie_id = re.search(r'.*code=([0-9]*)', test.select_one("dt.tit > a")["href"])[1]

In [69]:
# genre
genre = ",".join([_.text for _ in test.select_one("dd > dl.info_txt1 > dd").select("span > a")])

In [73]:
# vote_average
vote_average = test.select_one("dd > dl > dd > div > a > span.num").text

In [86]:
for current_movie in li_current_movie:
    movie = {}
    poster_path = current_movie.find("img")["src"].replace("m99_141_2", "f125") # resieze: m99_141_2 -> f125
    current_movie_dsc = current_movie.find("dl", attrs={"class":"lst_dsc"})
    if current_movie_dsc.select_one("dt.tit > span") is None :
        age = "전체"
    else :
        grade = current_movie_dsc.select_one("dt.tit > span").text
        if grade == "청소년 관람불가" :
            age = "청불"
        elif grade == "전체 관람가" :
            age = "전체"
        else : 
            age = re.search(r'[0-9]*', current_movie_dsc.select_one("dt.tit > span").text)[0]
    title = current_movie_dsc.select_one("dt.tit > a").text
    movie_id = re.search(r'.*code=([0-9]*)', current_movie_dsc.select_one("dt.tit > a")["href"])[1]
    genre = ",".join([_.text for _ in current_movie_dsc.select_one("dd > dl.info_txt1 > dd").select("span > a")])
    vote_average = current_movie_dsc.select_one("dd > dl > dd > div > a > span.num").text
    
    movie["title"] = title
    movie["genre_ids"] = genre
    movie["age"] = age
    movie["vote_average"] = vote_average
    movie["poster_path"] = poster_path
    movie["id"] = movie_id
    print(movie)
    print()

{'title': '담보', 'genre_ids': '드라마', 'age': '12', 'vote_average': '8.99', 'poster_path': 'https://movie-phinf.pstatic.net/20200929_240/1601370752754trKxu_JPEG/movie_image.jpg?type=f125', 'id': '183866'}

{'title': '국제수사', 'genre_ids': '액션,드라마,범죄', 'age': '15', 'vote_average': '4.32', 'poster_path': 'https://movie-phinf.pstatic.net/20200916_163/16002324767974gUSf_JPEG/movie_image.jpg?type=f125', 'id': '175393'}

{'title': '그린랜드', 'genre_ids': '액션,스릴러', 'age': '12', 'vote_average': '7.00', 'poster_path': 'https://movie-phinf.pstatic.net/20200929_185/1601357130092TtONR_JPEG/movie_image.jpg?type=f125', 'id': '195983'}

{'title': '테넷', 'genre_ids': '액션,SF', 'age': '12', 'vote_average': '8.27', 'poster_path': 'https://movie-phinf.pstatic.net/20200728_86/15959209130236RePy_JPEG/movie_image.jpg?type=f125', 'id': '190010'}

{'title': '극장판 포켓몬스터 뮤츠의 역습 EVOLUTION', 'genre_ids': '애니메이션', 'age': '전체', 'vote_average': '8.37', 'poster_path': 'https://movie-phinf.pstatic.net/20200902_168/15990280770197

{'title': '레이디 버드', 'genre_ids': '코미디,드라마', 'age': '15', 'vote_average': '8.51', 'poster_path': 'https://movie-phinf.pstatic.net/20180326_207/1522051022222Wrub8_JPEG/movie_image.jpg?type=f125', 'id': '158611'}

{'title': '브리짓 존스의 일기', 'genre_ids': '드라마,멜로/로맨스,코미디', 'age': '청불', 'vote_average': '8.68', 'poster_path': 'https://movie-phinf.pstatic.net/20111216_25/1324029438334jIoaL_JPEG/movie_image.jpg?type=f125', 'id': '31565'}

{'title': '블레이드 러너 2049', 'genre_ids': 'SF,액션', 'age': '15', 'vote_average': '7.99', 'poster_path': 'https://movie-phinf.pstatic.net/20170925_246/1506307175130N35Jq_JPEG/movie_image.jpg?type=f125', 'id': '88227'}

{'title': '사랑과 영혼', 'genre_ids': '멜로/로맨스,드라마', 'age': '15', 'vote_average': '9.20', 'poster_path': 'https://movie-phinf.pstatic.net/20171206_274/1512521901491RlovP_JPEG/movie_image.jpg?type=f125', 'id': '10173'}

{'title': '사랑하는 시바여 돌아오라', 'genre_ids': '', 'age': '전체', 'vote_average': '7.00', 'poster_path': 'https://movie-phinf.pstatic.net/20111221_104/

In [87]:
def get_current_movie_list() :
    result = []
    for current_movie in li_current_movie:
        movie = {}
        poster_path = current_movie.find("img")["src"].replace("m99_141_2", "f125") # resieze: m99_141_2 -> f125
        current_movie_dsc = current_movie.find("dl", attrs={"class":"lst_dsc"})
        if current_movie_dsc.select_one("dt.tit > span") is None :
            age = "전체"
        else :
            grade = current_movie_dsc.select_one("dt.tit > span").text
            if grade == "청소년 관람불가" :
                age = "청불"
            elif grade == "전체 관람가" :
                age = "전체"
            else : 
                age = re.search(r'[0-9]*', current_movie_dsc.select_one("dt.tit > span").text)[0]
        title = current_movie_dsc.select_one("dt.tit > a").text
        movie_id = re.search(r'.*code=([0-9]*)', current_movie_dsc.select_one("dt.tit > a")["href"])[1]
        genre = ",".join([_.text for _ in current_movie_dsc.select_one("dd > dl.info_txt1 > dd").select("span > a")])
        vote_average = current_movie_dsc.select_one("dd > dl > dd > div > a > span.num").text

        movie["title"] = title
        movie["genre_ids"] = genre
        movie["age"] = age
        movie["vote_average"] = vote_average
        movie["poster_path"] = poster_path
        movie["id"] = movie_id
        result.append(movie)
    return result

In [88]:
test = get_current_movie_list()
print(test)

[{'title': '담보', 'genre_ids': '드라마', 'age': '12', 'vote_average': '8.99', 'poster_path': 'https://movie-phinf.pstatic.net/20200929_240/1601370752754trKxu_JPEG/movie_image.jpg?type=f125', 'id': '183866'}, {'title': '국제수사', 'genre_ids': '액션,드라마,범죄', 'age': '15', 'vote_average': '4.32', 'poster_path': 'https://movie-phinf.pstatic.net/20200916_163/16002324767974gUSf_JPEG/movie_image.jpg?type=f125', 'id': '175393'}, {'title': '그린랜드', 'genre_ids': '액션,스릴러', 'age': '12', 'vote_average': '7.00', 'poster_path': 'https://movie-phinf.pstatic.net/20200929_185/1601357130092TtONR_JPEG/movie_image.jpg?type=f125', 'id': '195983'}, {'title': '테넷', 'genre_ids': '액션,SF', 'age': '12', 'vote_average': '8.27', 'poster_path': 'https://movie-phinf.pstatic.net/20200728_86/15959209130236RePy_JPEG/movie_image.jpg?type=f125', 'id': '190010'}, {'title': '극장판 포켓몬스터 뮤츠의 역습 EVOLUTION', 'genre_ids': '애니메이션', 'age': '전체', 'vote_average': '8.37', 'poster_path': 'https://movie-phinf.pstatic.net/20200902_168/1599028077019